In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving combined_matrix.csv to combined_matrix.csv


In [ ]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

In [ ]:
sorted_combined_matrix = pd.read_csv("combined_matrix.csv")

In [ ]:
sorted_combined_matrix

,parameter_location_id,2019-10-28T02:00:00+08:00,2018-05-28T12:00:00+08:00,2018-02-23T17:00:00+08:00,2018-03-25T14:00:00+08:00,2019-11-14T11:00:00+08:00,2021-06-01T17:00:00+08:00,2018-09-09T12:00:00+08:00,2018-12-04T01:00:00+08:00,2019-05-22T20:00:00+08:00,...,2019-08-13T15:00:00+08:00,2020-12-07T18:00:00+08:00,2021-01-15T20:00:00+08:00,2021-03-24T13:00:00+08:00,2018-08-17T02:00:00+08:00,2020-02-08T22:00:00+08:00,2019-04-18T07:00:00+08:00,2020-12-28T06:00:00+08:00,2020-04-14T11:00:00+08:00,2021-01-09T12:00:00+08:00
0,co_6168,1400.0,769.0,680.0,796.0,400.0,700.0,363.0,2000.0,800.0,...,400.0,800.0,1100.0,500.0,752.0,300.0,800.0,1100.0,500.0,500.0
1,no2_6168,73.0,65.0,44.0,49.0,55.0,22.0,14.0,45.0,97.0,...,22.0,63.0,89.0,36.0,12.0,17.0,80.0,68.0,46.0,29.0
2,o3_6168,49.0,78.0,76.0,113.0,63.0,243.0,130.0,21.0,199.0,...,182.0,49.0,49.0,91.0,51.0,60.0,137.0,37.0,131.0,53.0
3,pm10_6168,141.0,187.0,45.0,0.0,99.0,63.0,33.0,0.0,113.0,...,27.0,72.0,230.0,63.0,35.0,26.0,84.0,58.0,55.0,36.0
4,pm25_6168,80.0,57.0,29.0,103.0,32.0,24.0,8.0,25.0,29.0,...,10.0,70.0,87.0,14.0,27.0,22.0,13.0,82.0,46.0,18.0
5,so2_6168,10.0,14.0,5.0,15.0,7.0,8.0,3.0,3.0,4.0,...,4.0,2.0,16.0,8.0,7.0,4.0,5.0,1.0,9.0,8.0
6,co_6169,1400.0,809.0,905.0,630.0,500.0,600.0,566.0,1900.0,1000.0,...,600.0,800.0,1000.0,600.0,596.0,500.0,800.0,800.0,400.0,400.0
7,no2_6169,61.0,50.0,23.0,23.0,41.0,24.0,20.0,36.0,51.0,...,11.0,45.0,77.0,27.0,11.0,22.0,40.0,62.0,29.0,19.0
8,o3_6169,52.0,91.0,99.0,145.0,68.0,243.0,143.0,15.0,170.0,...,146.0,58.0,56.0,114.0,56.0,62.0,138.0,47.0,142.0,56.0
9,pm10_6169,105.0,202.0,0.0,65.0,60.0,83.0,42.0,0.0,141.0,...,39.0,67.0,203.0,61.0,38.0,28.0,59.0,72.0,68.0,32.0


In [ ]:
sorted_combined_matrix = sorted_combined_matrix.set_index('parameter_location_id')

sorted_combined_matrix_T =sorted_combined_matrix.T
sorted_combined_matrix_T.replace(to_replace=0, method='ffill', inplace=True)
sorted_combined_matrix_T.index

Index(['2019-10-28T02:00:00+08:00', '2018-05-28T12:00:00+08:00',
       '2018-02-23T17:00:00+08:00', '2018-03-25T14:00:00+08:00',
       '2019-11-14T11:00:00+08:00', '2021-06-01T17:00:00+08:00',
       '2018-09-09T12:00:00+08:00', '2018-12-04T01:00:00+08:00',
       '2019-05-22T20:00:00+08:00', '2020-05-07T23:00:00+08:00',
       ...
       '2019-08-13T15:00:00+08:00', '2020-12-07T18:00:00+08:00',
       '2021-01-15T20:00:00+08:00', '2021-03-24T13:00:00+08:00',
       '2018-08-17T02:00:00+08:00', '2020-02-08T22:00:00+08:00',
       '2019-04-18T07:00:00+08:00', '2020-12-28T06:00:00+08:00',
       '2020-04-14T11:00:00+08:00', '2021-01-09T12:00:00+08:00'],
      dtype='object', length=25941)

In [ ]:
sorted_combined_matrix_T

parameter_location_id,co_6168,no2_6168,o3_6168,pm10_6168,pm25_6168,so2_6168,co_6169,no2_6169,o3_6169,pm10_6169,...,o3_6274,pm10_6274,pm25_6274,so2_6274,co_6273,no2_6273,o3_6273,pm10_6273,pm25_6273,so2_6273
2019-10-28T02:00:00+08:00,1400.0,73.0,49.0,141.0,80.0,10.0,1400.0,61.0,52.0,105.0,...,109.0,112.0,60.0,15.0,600.0,60.0,109.0,122.0,75.0,13.0
2018-05-28T12:00:00+08:00,769.0,65.0,78.0,187.0,57.0,14.0,809.0,50.0,91.0,202.0,...,176.0,182.0,41.0,18.0,730.0,14.0,190.0,181.0,73.0,19.0
2018-02-23T17:00:00+08:00,680.0,44.0,76.0,45.0,29.0,5.0,905.0,23.0,99.0,202.0,...,111.0,68.0,36.0,18.0,624.0,24.0,109.0,74.0,43.0,13.0
2018-03-25T14:00:00+08:00,796.0,49.0,113.0,45.0,103.0,15.0,630.0,23.0,145.0,65.0,...,261.0,176.0,117.0,16.0,1096.0,30.0,243.0,156.0,124.0,26.0
2019-11-14T11:00:00+08:00,400.0,55.0,63.0,99.0,32.0,7.0,500.0,41.0,68.0,60.0,...,85.0,102.0,49.0,7.0,400.0,30.0,82.0,117.0,64.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-08T22:00:00+08:00,300.0,17.0,60.0,26.0,22.0,4.0,500.0,22.0,62.0,28.0,...,73.0,87.0,68.0,7.0,900.0,20.0,70.0,78.0,71.0,5.0
2019-04-18T07:00:00+08:00,800.0,80.0,137.0,84.0,13.0,5.0,800.0,40.0,138.0,59.0,...,148.0,79.0,45.0,11.0,900.0,31.0,152.0,72.0,50.0,15.0
2020-12-28T06:00:00+08:00,1100.0,68.0,37.0,58.0,82.0,1.0,800.0,62.0,47.0,72.0,...,104.0,216.0,154.0,12.0,900.0,31.0,56.0,72.0,50.0,15.0
2020-04-14T11:00:00+08:00,500.0,46.0,131.0,55.0,46.0,9.0,400.0,29.0,142.0,68.0,...,156.0,85.0,44.0,5.0,400.0,18.0,151.0,87.0,56.0,9.0


In [ ]:
# Normalize the combined_matrix data
normalized_data = sorted_combined_matrix_T
scaler = StandardScaler()
normalized_data = scaler.fit_transform(normalized_data)

In [ ]:
normalized_data

array([[ 2.0656474 ,  0.95691022, -0.86911713, ...,  0.49745027,
         0.55010976,  0.09917771],
       [ 0.20562045,  0.66630627, -0.3036943 , ...,  1.32893658,
         0.50111564,  0.46944021],
       [-0.05672883, -0.09652911, -0.34268898, ..., -0.17901317,
        -0.2337963 ,  0.09917771],
       ...,
       [ 1.18132397,  0.77528275, -1.10308519, ..., -0.20719914,
        -0.06231685,  0.22259854],
       [-0.58732289, -0.02387812,  0.72966467, ...,  0.00419568,
         0.08466554, -0.14766396],
       [-0.58732289, -0.64141152, -0.79112777, ...,  0.15921855,
        -0.01332272,  0.09917771]])

In [ ]:
# Convert normalized_data to a NumPy array
data_array = np.array(normalized_data)

In [ ]:
data_array

array([[ 2.0656474 ,  0.95691022, -0.86911713, ...,  0.49745027,
         0.55010976,  0.09917771],
       [ 0.20562045,  0.66630627, -0.3036943 , ...,  1.32893658,
         0.50111564,  0.46944021],
       [-0.05672883, -0.09652911, -0.34268898, ..., -0.17901317,
        -0.2337963 ,  0.09917771],
       ...,
       [ 1.18132397,  0.77528275, -1.10308519, ..., -0.20719914,
        -0.06231685,  0.22259854],
       [-0.58732289, -0.02387812,  0.72966467, ...,  0.00419568,
         0.08466554, -0.14766396],
       [-0.58732289, -0.64141152, -0.79112777, ...,  0.15921855,
        -0.01332272,  0.09917771]])

In [ ]:
# Prepare different hour3 windows from the dataset
window_sizes = [3, 6, 12, 24, 48, 96, 192, 384, 768]
mse_values = []

In [ ]:
import keras
from keras import layers
import numpy as np
from keras.callbacks import EarlyStopping

def create_nested_FGM_VAE(input_dim, encoding_dims, latent_dim, dropout_rate=0.2):
    # Define the encoder model
    input_img = keras.Input(shape=(input_dim,))
    x = input_img

    encoder_layers = []

    for encoding_dim in encoding_dims:
        encoder_layer = layers.Dense(encoding_dim, activation='swish')(x)
        x = encoder_layer
        encoder_layer = layers.Dropout(dropout_rate)(encoder_layer)  # Add dropout here
        encoder_layers.append(encoder_layer)

    encoder = keras.Model(input_img, encoder_layers[-1], name='encoder')

    # Define the latent space layer
    z_mean = layers.Dense(latent_dim, name='z_mean')(encoder_layers[-1])
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(encoder_layers[-1])

    # Define the reparameterization trick to sample from the latent space
    def sampling(args):
        z_mean, z_log_var = args
        epsilon = keras.backend.random_normal(shape=(keras.backend.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0)
        return z_mean + keras.backend.exp(0.5 * z_log_var) * epsilon

    z = layers.Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # Define the decoder model
    encoded_input = keras.Input(shape=(encoding_dims[-1],))  # Use the last encoding dimension
    x = encoded_input

    for encoding_dim in reversed(encoding_dims):
        decoder_layer = layers.Dense(encoding_dim, activation='swish')(x)
        x = decoder_layer
        decoder_layer = layers.Dropout(dropout_rate)(decoder_layer)  # Add dropout here

    decoded = layers.Dense(input_dim, activation='linear')(x)

    decoder = keras.Model(encoded_input, decoded, name='decoder')

    # Define the Variational Autoencoder (VAE) model
    vae = keras.Model(input_img, decoder(encoder(input_img)), name='vae')

    return vae, encoder, decoder

In [ ]:
# Define the function to train and evaluate the autoencoder model for a given window size
def train_and_evaluate_model(window_size,latent_dim):
    windows = np.array([data_array[i:i + window_size].flatten() for i in range(data_array.shape[0] - window_size + 1)])
    num_inputs = windows.shape[1]

    input_dim = num_inputs
    encoding_dims = [1024, 512, 256]  # Example: 3 layers with encoding dimensions

    # Create the nested FGM-VAE architecture
    vae, encoder, decoder = create_nested_FGM_VAE(input_dim, encoding_dims, latent_dim)

    # Compile the autoencoder model
    vae.compile(optimizer=keras.optimizers.Adam(), loss='mse')

    # Define the split points for training, validation, and testing
    train_split = 0.8  # 80% for training
    val_split = 0.1  # 10% for validation
    test_split = 0.1  # 10% for testing

    # Calculate the indices for splitting the data
    total_length = len(windows)
    train_index = int(total_length * train_split)
    val_index = train_index + int(total_length * val_split)

    # Split the data into training, validation, and testing sets
    train_data = windows[:train_index]
    val_data = windows[train_index:val_index]
    test_data = windows[val_index:]

    # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the autoencoder with the training and validation data
    history = vae.fit(
        train_data, train_data,
        epochs=100,
        batch_size=256,
        shuffle=True,
        validation_data=(val_data, val_data),
        callbacks=[early_stopping]
    )

    # Display the autoencoder model summary
    vae.summary()

    # Initialize an array to store MSE values for each window's nth hour reconstruction
    mse_values = []
    num_inputs_per_hour = data_array.shape[1]
    input_width = window_size * num_inputs_per_hour
    # Iterate over each window in the test data
    for window in test_data:
        # Copy the window for modification
        window_mod = window.copy()
        # Zero out the third hour of each window
        window_mod[(window_size-1) * num_inputs_per_hour:window_size * num_inputs_per_hour] = 0
        # Reshape the modified window to match the input shape
        window_mod = window_mod.reshape((1, input_width))
        # Reconstruct the modified window
        reconstructed_window = vae.predict(window_mod)
        # Calculate MSE between the original nth hour and the reconstructed nth hour
        mse = np.mean((window[(window_size-1) * num_inputs_per_hour:window_size * num_inputs_per_hour] - reconstructed_window[0, (window_size-1) * num_inputs_per_hour:window_size * num_inputs_per_hour]) ** 2)
        # Append MSE value to the list
        mse_values.append(mse)

    # Calculate the average MSE across all windows
    average_mse = np.mean(mse_values)
    print("Average MSE for the " + str(window_size) + " hour reconstruction on test data:", average_mse)
    return average_mse

In [ ]:
# Iterate through the different window sizes and train/evaluate the models
for window_size in window_sizes:
    latent_dim = 32
    # Train and evaluate the FGM-VAE model with a specified window size and latent dimension
    average_mse = train_and_evaluate_model(window_size, latent_dim)
    mse_values.append(average_mse)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 20ms/step
Average MSE for the 384 hour reconstruction on test data: 1.1116464741495853
Epoch 1/100
79/79 [==============================] - 6s 51ms/step - loss: 0.9979 - val_loss: 0.9347
Epoch 2/100
79/79 [==============================] - 2s 28ms/step - loss: 0.9890 - val_loss: 0.9308
Epoch 3/100
79/79 [==============================] - 2s 29ms/step - loss: 0.9830 - val_loss: 0.9268
Epoch 4/100
79/79 [==============================] - 2s 29ms/step - loss: 0.9762 - val_loss: 0.9228
Epoch 5/100
79/79 [==============================] - 2s 29ms/step - loss: 0.9699 - val_loss: 0.9192
Epoch 6/100
79/79 [==============================] - 2s 29ms/step - loss: 0.9644 - val_loss: 0.9163
Epoch 7/100
79/79 [==============================] - 2s 29ms/step - loss: 0.9598 - val_loss: 0.9141
Epoch 8/100
79/79 [==============================] - 2s 29ms/step - loss: 0.9559 - val_loss: 0.9122
Epoch 9/100
79/79 [=

In [ ]:
# Determine the window size with the lowest MSE
best_window_size = window_sizes[np.argmin(mse_values)]
lowest_mse = np.min(mse_values)
print("Best Window Size:", best_window_size)
print("Lowest MSE:", lowest_mse)

Best Window Size: 3
Lowest MSE: 1.0552936599416205


In [ ]:
import plotly.graph_objects as go

# Create string labels for the window sizes
window_labels = [str(window_size) + "-hour" for window_size in window_sizes]

# Define custom colors for each bar
custom_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22']

# Define custom widths for each bar
custom_widths = [0.3, 0.3, 0.3, 0.3, 0.3,0.3, 0.3, 0.3, 0.3]

# Plot the MSE values on a bar chart with custom colors and widths
fig = go.Figure(data=[go.Bar(x=window_labels, y=mse_values, marker_color=custom_colors, width=custom_widths)])
fig.update_layout(
    title="Average MSE for Different Window Sizes",
    xaxis_title="Window Size",
    yaxis_title="Average MSE",
    showlegend=False,
    height=800,  # Set the height of the figure
    width=1000   # Set the width of the figure
)
fig.show()

print("Best Window Size:", best_window_size)
print("Lowest MSE:", lowest_mse)


Best Window Size: 3
Lowest MSE: 1.0552936599416205


In [ ]:
mse_values

[1.0552936599416205,
 1.0700908391450032,
 1.0801573702801948,
 1.0953998850728386,
 1.1149653694737955,
 1.1327426417070805,
 1.1054087960317278,
 1.1116464741495853,
 1.078694533033546]